#### Copied this code lmao
The code uses numpy so I need to change it to pytorch
For that scroll down until you find it

In [ ]:
def compute_perturbation(model, x1, label_x0, label_k, val_grad):
    # Get gradient and function value for class k
    gradient_k, fx1 = grad_val(model, x1, label_k)
    
    # Calculate the difference between gradients
    gradient_diff = [g - g0 for g, g0 in zip(gradient_k, val_grad)]
    gradient_diff_norm = np.sqrt(np.sum([np.linalg.norm(g)**2 for g in gradient_diff]))
    
    # Compute the function value difference for class k
    f_k = fx1[0, label_k] - fx1[0, label_x0]
    
    # Compute the weighted gradient ratio
    fk_wk = np.linalg.norm(f_k) / (gradient_diff_norm + 1e-3)
    
    return gradient_diff, fk_wk, f_k

def deep_fool_algorithm(model, x0, step=0.01, max_iter=15):
    # Get the initial prediction and label of x0
    preds = model(x0)[0]
    initial_label = np.argmax(preds)
    x1 = deepcopy(x0)
    current_label = initial_label
    loop_iter = 0
    perturbations = []

    while current_label == initial_label and loop_iter < max_iter:
        perturb_direction = [np.zeros_like(x_input) for x_input in x0]
        perturb_magnitude = 0
        best_l = np.inf
        
        # Get the gradient and function value for the initial label
        val_grad, fx1 = grad_val(model, x1, initial_label)
        
        # Iterate over all classes except the true label
        for label_k in range(10):
            if label_k == initial_label:
                continue

            # Compute perturbation for the current class label
            perturb_diff, fk_wk, f_k = compute_perturbation(model, x1, initial_label, label_k, val_grad)
            
            # Update perturbation if the current perturbation direction is better
            if fk_wk < best_l:
                perturb_direction, perturb_magnitude = perturb_diff, f_k
                best_l = fk_wk
        
        # Normalize the perturbation
        perturb_direction_norm_squared = np.sum([np.linalg.norm(w_l_input)**2 for w_l_input in perturb_direction])
        perturb_magnitude_norm = np.linalg.norm(perturb_magnitude)
        perturb_constant = perturb_magnitude_norm / (perturb_direction_norm_squared + 1e-3)
        
        # Scale the perturbation by the calculated constant
        perturbation = [perturb_constant * w_l_input for w_l_input in perturb_direction]
        perturbations.append(perturbation)

        # Update x1 with the new perturbation
        x1 = [x1_item + (1 + step) * perturb_item for x1_item, perturb_item in zip(x1, perturbation)]
        x1 = np.array(x1)

        # Get the new prediction for x1
        preds = model(x1)[0]
        current_label = np.argmax(preds)
        loop_iter += 1

    # Sum up all perturbations for the final result
    total_perturbation = [np.zeros_like(x_input) for x_input in x0]
    for i in range(len(x0)):
        for perturb in perturbations:
            total_perturbation[i] += perturb[i][0]

    return total_perturbation, loop_iter, current_label

### This is the pytorch code brah

In [ ]:
import torch
from models.cnn import AudioCNN
from utils import read_config
import torch.nn as nn

##### Helper Functions for DeepFool


In [ ]:
def compute_jacobian_and_values(model, x, lengths):
    """
    Computes the function values (logits) and the Jacobian (gradients)
    for all classes w.r.t the input x.
    
    This function replaces 'grad_val' from the implementation above and performs the
    gradient calculation required.
    
    Args:
        model (nn.Module): The target neural network model.
        x (torch.Tensor): The input tensor (requires_grad=True).
        
    Returns:
        tuple: (f_x, jacobian_dict), where f_x is the logit tensor, and
               jacobian_dict maps class index to its gradient tensor.
    """
    # 1. Compute logits
    f_x = model(x.unsqueeze(0), lengths) # Add batch dimension
    f_x = f_x.squeeze(0)        # Remove batch dimension for 1D logits

    # 2. Get the number of classes
    num_classes = f_x.numel()
    
    # 3. Create a dictionary to hold the Jacobian (gradients) for each class
    jacobian_dict = {}

    # 4. Iterate over each class to compute its gradient w.r.t. x
    for k in range(num_classes):
        # Zero out the gradients of the model parameters
        model.zero_grad()
        
        # Select the logit for class k
        f_k = f_x[k]
        
        # Compute gradient of f_k w.r.t. the input x
        # This will populate x.grad if retain_graph=False, but we use torch.autograd.grad
        # to explicitly compute the gradient for f_k without side effects
        gradient_k = torch.autograd.grad(f_k, x, retain_graph=True)[0]
        
        jacobian_dict[k] = gradient_k
        
    return f_x, jacobian_dict

#### 2. DeepFool Implementation


In [ ]:

def deep_fool_attack(model, x_original, lengths, step=0.01, max_iter=50, overshoot=0.02, num_classes=10, device='cpu'):
    """
    Performs the DeepFool adversarial attack.
    
    Args:
        model (nn.Module): The target neural network model.
        x_original (torch.Tensor): The original input data (single sample).
        step (float): Step size for moving towards the boundary (redundant with overshoot).
        max_iter (int): Maximum number of iterations for the attack.
        overshoot (float): Parameter to push the perturbation slightly past the boundary.
        num_classes (int): Total number of classes.
        device (str): Device (e.g., 'cpu', 'cuda').
        
    Returns:
        tuple: (r_total, loop_iter, final_label), where r_total is the adversarial
               perturbation, loop_iter is the number of steps taken, and final_label
               is the misclassified label.
    """
    
    model.eval()
    
    # Ensure input is on the correct device, requires gradient tracking, and is a single sample
    x_i = x_original.clone().detach().to(device)
    x_i.requires_grad = True
    
    # 1. Initial prediction
    logits_original = model(x_i.unsqueeze(0), lengths).squeeze(0)
    initial_label = logits_original.argmax().item()
    current_label = initial_label
    
    r_total = torch.zeros_like(x_i).to(device)
    loop_iter = 0

    # Main iterative loop
    while current_label == initial_label and loop_iter < max_iter:
        
        # Compute function values (f_k) and gradients (w_k) for the current x_i
        f_x_i, w_k_dict = compute_jacobian_and_values(model, x_i, lengths)
        
        # Get the gradient for the true class (w_true)
        w_true = w_k_dict[initial_label]
        f_true = f_x_i[initial_label]
        
        min_w_norm = float('inf')
        min_r_l = None
        
        # 2. Iterate over all other classes (k != true) to find the nearest boundary
        for k in range(num_classes):
            if k == initial_label:
                continue

            w_k = w_k_dict[k]
            f_k = f_x_i[k]
            
            # Compute the vector w_l = w_k - w_true and the value f_l = f_k - f_true
            # This defines the separating hyperplane w_l * x + f_l = 0
            w_l = w_k - w_true
            f_l = f_k - f_true
            
            # The distance from the current point x_i to the hyperplane is:
            # distance = |f_l| / ||w_l||_2
            # The perturbation r_l is the projection of x_i onto this hyperplane:
            # r_l = - (f_l / ||w_l||_2^2) * w_l
            
            w_l_norm_sq = w_l.pow(2).sum()
            
            # Avoid division by zero
            if w_l_norm_sq.item() < 1e-8:
                continue
                
            w_norm = w_l_norm_sq.sqrt()
            
            # Compute the perturbation magnitude and direction
            r_l = - (f_l.abs() / w_l_norm_sq) * w_l
            
            # Check if this boundary is closer
            if f_l.abs() / w_norm < min_w_norm:
                min_w_norm = f_l.abs() / w_norm
                min_r_l = r_l
        
        # 3. Update the total perturbation and the input
        if min_r_l is not None:
            r_total += min_r_l
            
            # The step is applied to the final result, including the overshoot
            x_i_new = x_original.clone().detach().to(device) + (1 + overshoot) * r_total
            x_i = x_i_new.clone().detach().to(device)
            x_i.requires_grad = True # Must re-enable gradient tracking
            
            # 4. Check the new prediction
            with torch.no_grad():
                current_label = model(x_i.unsqueeze(0), lengths).squeeze(0).argmax().item()
        else:
            # Should not happen if the model has multiple classes
            break 
            
        loop_iter += 1

    return (1 + overshoot) * r_total, loop_iter, current_label


In [ ]:
# 1. Initialize the model

config = read_config("../config/cnn.json")

device = AudioCNN.setDevice()

model = AudioCNN.loadModel(path="model.pt", config=config, device=device)

# 2. Create a mock input audio sample (e.g., 1-second at 22050 Hz)
mock_audio_sample = torch.randn(22050, device=device) 
lengths = torch.tensor([22050], device=device)

# 3. Get initial prediction

with torch.no_grad():
    initial_logits = model(mock_audio_sample.unsqueeze(0), lengths)  # ← FIX THIS
    initial_label = initial_logits.argmax().item()  # ← ADD THIS
print(f"Initial prediction: Class {initial_label}")

criterion = nn.CrossEntropyLoss()

# 4. Run the DeepFool attack
print("Running DeepFool attack (Max Iterations: 50)...")
r_total, loop_iter, final_label = deep_fool_attack(
    model=model, 
    x_original=mock_audio_sample, 
    lengths=lengths,
    max_iter=50, 
    overshoot=0.01,
    num_classes=10,
    device=device
)

# 5. Create adversarial example
x_adv = mock_audio_sample + r_total

# 6. Verify final prediction
with torch.no_grad():
    final_logits = model(x_adv.unsqueeze(0), lengths)
    final_label_check = final_logits.argmax().item()

# 7. Report results
print("-" * 30)
if initial_label != final_label_check:
    print(f"Attack successful!")
    print(f"Iterations: {loop_iter}")
    print(f"Original Label: {initial_label}")
    print(f"Adversarial Label: {final_label_check}")
    
    # Calculate perturbation magnitude (L2 norm)
    r_norm = r_total.norm().item()
    x_norm = mock_audio_sample.norm().item()
    print(f"Relative Perturbation (L2/L2): {r_norm / x_norm:.4e}")
else:
    print("Attack failed or max iterations reached without misclassification.")
    print(f"Iterations: {loop_iter}")
    print(f"Final Label: {final_label_check} (Same as original)")



